# Model Validation

In [1]:
# Import required libraries
import numpy as np
import pandas as pd
import re

from gensim.models import Word2Vec
from sklearn.metrics.pairwise import cosine_similarity
import mlflow

import json
import requests
import boto3

import streamlit as st

import warnings
warnings.filterwarnings("ignore")

### Logging

In [55]:
model_path = "word2vec.model"
recipe_path = "processed_cookbook.pkl"

mlflow.set_tracking_uri("http://127.0.0.1:5000")
mlflow.set_experiment("word2vec_experiment")

with mlflow.start_run():
    # mlflow.log_params(params)

    model_info = mlflow.pyfunc.log_model(
        name="word2vec_model",
        python_model="./models/word2vec.py",
        pip_requirements=["gensim==4.3.3",
                          "json"] # json version?        
    )
    
    # Validation 
    

2025/07/19 19:51:22 INFO mlflow.tracking.fluent: Experiment with name 'word2vec_experiment' does not exist. Creating a new experiment.


🏃 View run bright-goose-311 at: http://127.0.0.1:5000/#/experiments/505961469723674826/runs/1253ef6b216948a6b7bec142fea2a953
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/505961469723674826


In [49]:
model = mlflow.pyfunc.load_model(model_info._model_uri)

In [ ]:
def connect_database():
    """Connect to the DynamoDB database."""
    dynamodb = boto3.resource("dynamodb",
                            aws_access_key_id=st.secrets.s3.AWS_ACCESS_KEY_ID,
                            aws_secret_access_key=st.secrets.s3.AWS_SECRET_ACCESS_KEY,
                            region_name=st.secrets.s3.AWS_DEFAULT_REGION)
    table = dynamodb.Table(st.secrets.s3.DB_NAME)
    return table

table = connect_database()

user_id = 110833230122006731136
user_item = table.get_item(Key={"user_id": int(user_id)})["Item"]

liked_idx = list(map(int, user_item["liked_idx"].keys()))
disliked_idx = list(map(int, user_item["disliked_idx"].keys()))

loaded_model = mlflow.pyfunc.load_model(model_uri=model_info.model_uri)
predicted = loaded_model.predict([liked_idx, disliked_idx])

In [ ]:
np.asarray(json.loads(predicted))

array([2576, 5757, 4283, 6044, 4242, 4181, 3195, 3736, 4375, 6086])